In [1]:
import os
import copy
import sys
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torchvision.utils import save_image

from model import VGG16_BN
import utils
import data

# args = utils.get_argument()
device = 'cpu'
# Get benign model
logging.info("Loading model..")
model = VGG16_BN()
chk = torch.load("checkpoint/benign.pth.tar", map_location=device)
model.load_state_dict(chk)
model = model.to(device)

# Make target dataset
logging.info("Loading data..")
_, dataset = data.get_data()

target_name = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
target_class = 0
# target_class : target misclassification class
logging.info("Loading target data")
# loader, name = utils.load_target_loader(dataset, target_class)
target_idx = [i for i in range(len(dataset)) if dataset[i][1] == target_class]
target_dataset = Subset(dataset, target_idx)
target_loader = DataLoader(target_dataset, batch_size=500, num_workers=4, pin_memory=True)
logging.info("Loaded data for %s" % target_name[target_class])

# Select Neuron
selected_neuron, target_activation = utils.select_neuron(1, model, target_loader, device)

Files already downloaded and verified


In [ ]:
del dataset, target_idx, target_dataset
print(selected_neuron, target_activation)

Error: Kernel is dead

In [ ]:
import utils
# Trigger Formation
mask_loc = 1
logging.info("Generating trigger for %s", target_name[target_class])
base = torch.ones(1,3,32,32, requires_grad=False)
mask = utils.generate_mask((1,3,32,32), loc=mask_loc)
# trigger = generate_mask((1,3,32,32), loc=mask_loc)

# mask.requires_grad = False
trigger = (base*mask)
trigger.requires_grad = True
# optimizer = torch.optim.SGD([trigger], lr=1e-3)
# Using gradient descent for trigger formation
eps = 0.1
model.train()

x, y = utils.get_trigger_offset(mask_loc)

for iter in range(10):
    activation = model(trigger, get_activation=1, neuron=selected_neuron)
    activation = activation.squeeze(0)
    target = torch.ones(activation.size(), device=device) * target_activation

    loss = F.mse_loss(activation, target)
    # loss = (target - activation)
    if iter % 100 == 0:
        logging.info("Loss : {}".format(loss.item()))

    if loss.item() < 1e-5:
        logging.info("Converged")
        break

    # model.zero_grad()
    loss.backward(retain_graph=True)
    trigger.retain_grad()
    trigger_grad = trigger.grad.data
    print(torch.sum(trigger_grad[:,:,x:x+8, y:y+9]))
    # optimizer.step()
    # print(trigger[:,:,x:x+8, y:y+9])
    trigger = trigger + eps*trigger_grad
    # print(trigger[:,:,x:x+8, y:y+9])
    # break
    # trigger = trigger*mask
    trigger = torch.clamp(trigger, 0, 1)

    trigger = trigger.detach()
    trigger.requires_grad = True
    # print(trigger[:,:,x:x+8, y:y+9])

In [ ]:
del base, mask, trigger, loss
model.zero_grad()